# Assessment

<p style='text-align: justify;'>
Congratulations on finishing the bio-inspired course!! Hopefully, you learned some valuable skills along the way and had fun doing it. Now it is time to put those skills to the test. In this assessment, we have the following problem: Soccer passing strategies. First, you will make the effectiveness of this approach depends on technical skill, tactical understanding, and coordination between players. Let's get started!
</p>

## Problem: Soccer passing strategies

<p style='text-align: justify;'>
Soccer passing strategies, especially those that move the ball from goalkeeper to striker quickly and efficiently without a long shot or <i>hot link</i>, are critical in the modern game. The idea is to maintain possession of the ball, moving it across the pitch with a series of short, controlled passes, often involving midfielders and defenders. From the following graph, find the most efficient passing path leaving the goalkeeper (1) and arriving at the last attacker (11), considering the possible connections and distances between players on the same team.
</p>    

<p style="text-align: center;">
<img src="./../images/figure16_soccer_assessment.jpg" style="width: 500px;">        
</p>


### Characteristics of the connections and distances

We encourage you to start with a model pretrained from the following table. Load the model with the correct values. Remember that the features have two dimensions: **player origin**, **player destination**, and **distance**. In summary, your model will require an output layer of one bio-inspired algorithm. 

| Player origin | Player destination  | Distance (m)
| :-:           | :-:                 | :-: 
|1 				| 3  				  | 4
|1 				| 5  				  | 6
|1 				| 4  				  | 4
|1 				| 7  				  | 20
|3 				| 2  				  | 4
|4 				| 2  				  | 6
|4 				| 5  				  | 4
|2 				| 5  				  | 5
|5 				| 6  				  | 4
|5 				| 7  				  | 8
|6 				| 9  				  | 5
|7 				| 8  				  | 6
|7 				| 11 				  | 15
|7 				| 9  				  | 7
|8 				| 10 				  | 7
|8 				| 9  				  | 15 
|9 				| 10 				  | 10
|9 				| 11 				  | 8  
|10             | 11 				  | 6

`Our mission is to help the coach create the best pass-to-goal strategy, that is, to find the ideal order, with the shortest path, to pass the ball between the players until it reaches the opposing goal`. To do that, answer the following item: 

a) Implement a bio-inspired algorithm to find the best route between the players using soccer passing strategy.

## ☆ Solutions ☆ 

### `GA`

### ⊗ **Importing libraries into our algorithm**

In [73]:
import random

### ⊗ **Players connections**: 

Let's define all possible connections between players

In [74]:
player_connections = {
    "1": {"3": 4, "4": 4, "5": 6, "7": 20},
    "2": {"4": 6, "5": 5},
    "3": {"1": 4, "2": 4},
    "4": {"1": 4, "2": 6, "5": 4},
    "5": {"1": 6, "2": 5, "4": 4, "6": 4, "7": 8},
    "6": {"5": 4, "9": 5},
    "7": {"1": 20, "5": 8, "8": 6, "9": 7, "11": 15},
    "8": {"7": 6, "9": 15, "10": 7},
    "9": {"6": 5, "7": 7, "8": 15, "10": 10, "11": 8},
    "10": {"8": 7, "9": 10, "11": 6},
    "11": {"7": 15, "9": 8, "10": 6}
}

### ⊗ **Parameters**: 

Here we will define the initial parameters for our genetic algorithm

In [75]:
POPULATION_SIZE = 500
NUM_GENERATIONS = 10000
MUTATION_RATE = 0.3

### ⊗ **Fitness**: 

At this point we will create our fitness function that will evaluate the best routes among players

In [76]:
def calculate_fitness(route):
    total_distance = 0
    for i in range(len(route) - 1):
        current_player = route[i]
        next_player = route[i+1]
        if current_player in player_connections and next_player in player_connections[current_player]:
            total_distance += player_connections[current_player][next_player]
        else:
            return float('inf')
    return total_distance

### ⊗ **Population**: 

In [77]:
def create_initial_population(players, size):
    population = []
    for _ in range(size):
        route = ["1"] + random.sample(players[1:-1], random.randint(2, len(players) - 3)) + ["11"]
        population.append(route)
    return population

### ⊗ **Crossover**: 

Now we are going to perform the crossing between the best passes (routes) between the players.

In [78]:
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 2)
    child1 = parent1[:crossover_point] + [player for player in parent2 if player not in parent1[:crossover_point]]
    child2 = parent2[:crossover_point] + [player for player in parent1 if player not in parent2[:crossover_point]]
    return child1, child2

### ⊗ **Mutate**: 

Here a small change will be made in the moves in order to vary some indexes of the routes in order to introduce variability.

In [79]:
def mutate(individual):
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(1, len(individual) - 1), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

### ⊗ **Run genetic algorithm**: 

Let's run the genetic algorithm so we can figure out the best move to make.

In [80]:
def genetic_algorithm():
    players = list(player_connections.keys())
    population = create_initial_population(players, POPULATION_SIZE)

    for generation in range(NUM_GENERATIONS):
        population = sorted(population, key=lambda x: calculate_fitness(x))
        new_population = population[:POPULATION_SIZE // 2]
        
        while len(new_population) < POPULATION_SIZE:
            parent1, parent2 = random.choices(population, k=2)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1)
            mutate(child2)
            new_population.extend([child1, child2])

        population = new_population

    best_route = min(population, key=lambda x: calculate_fitness(x))
    shortest_distance = calculate_fitness(best_route)
    
    return best_route, shortest_distance

best_route, shortest_distance = genetic_algorithm()
print("Best route:", best_route)
print("Best distance:", shortest_distance)

Best route: ['1', '5', '6', '9', '11']
Best distance: 23


### ⊗ **Optimizing the genetic algorithm using Intel® SigOpt**: 

In [81]:
import random
import sigopt

player_connections = {
    "1": {"3": 4, "4": 4, "5": 6, "7": 20},
    "2": {"4": 6, "5": 5},
    "3": {"1": 4, "2": 4},
    "4": {"1": 4, "2": 6, "5": 4},
    "5": {"1": 6, "2": 5, "4": 4, "6": 4, "7": 8},
    "6": {"5": 4, "9": 5},
    "7": {"1": 20, "5": 8, "8": 6, "9": 7, "11": 15},
    "8": {"7": 6, "9": 15, "10": 7},
    "9": {"6": 5, "7": 7, "8": 15, "10": 10, "11": 8},
    "10": {"8": 7, "9": 10, "11": 6},
    "11": {"7": 15, "9": 8, "10": 6}
}

def calculate_fitness(route):
    total_distance = 0
    for i in range(len(route) - 1):
        current_player = route[i]
        next_player = route[i+1]
        if current_player in player_connections and next_player in player_connections[current_player]:
            total_distance += player_connections[current_player][next_player]
        else:
            return float(999999)
    return total_distance

def create_initial_population(players, size):
    population = []
    for _ in range(size):
        route = ["1"] + random.sample(players[1:-1], random.randint(2, len(players) - 3)) + ["11"]
        population.append(route)
    return population

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 2)
    child1 = parent1[:crossover_point] + [player for player in parent2 if player not in parent1[:crossover_point]]
    child2 = parent2[:crossover_point] + [player for player in parent1 if player not in parent2[:crossover_point]]
    return child1, child2

def mutate(individual,mutation_rate):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(1, len(individual) - 1), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

def genetic_algorithm(pop_size, num_generations, mutation_rate):
    players = list(player_connections.keys())
    population = create_initial_population(players, pop_size)

    for generation in range(num_generations):
        population = sorted(population, key=lambda x: calculate_fitness(x))
        new_population = population[:pop_size // 2]
        
        while len(new_population) < pop_size:
            parent1, parent2 = random.choices(population, k=2)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1,mutation_rate)
            mutate(child2,mutation_rate)
            new_population.extend([child1, child2])

        population = new_population

    best_route = min(population, key=lambda x: calculate_fitness(x))
    shortest_distance = calculate_fitness(best_route)
    
    return best_route, shortest_distance

In [82]:
def evaluate_genetic_algorithm(params):
    pop_size = params['pop_size']
    num_generations = params['num_generations']
    mutation_rate = params['mutation_rate']

    best_route, shortest_distance = genetic_algorithm(pop_size, num_generations, mutation_rate)
    return shortest_distance

def sigopt_optimization():
    SIGOPT_API_TOKEN = 'MJZRDBRNTMOPVYPCTTMRAMDIQDUEDTDJGOUMPFSIQTRKLYJJ'

    conn = sigopt.Connection(client_token=SIGOPT_API_TOKEN)

    param_space = [
        {'name': 'pop_size', 'type': 'int', 'bounds': {'min': 100, 'max': 1000}},
        {'name': 'num_generations', 'type': 'int', 'bounds': {'min': 100, 'max': 4000}},
        {'name': 'mutation_rate', 'type': 'double', 'bounds': {'min': 0.01, 'max': 0.5}},
    ]

    experiment = conn.experiments().create(
        name='GA Optimization',
        parameters=param_space,
        metrics=[{'name': 'shortest_distance', 'objective': 'minimize'}]
    )

    for _ in range(30):
        suggestion = conn.experiments(experiment.id).suggestions().create()
        suggestion_id = suggestion.id
        suggested_params = suggestion.assignments
        result = evaluate_genetic_algorithm(suggested_params)
        conn.experiments(experiment.id).observations().create(
            suggestion=suggestion_id,
            value=result,
        )

    best_assignments = conn.experiments(experiment.id).best_assignments().fetch()

    print("Best hyperparameters found:")
    print(best_assignments.data[0].assignments)

    best_params = best_assignments.data[0].assignments
    best_route, shortest_distance = genetic_algorithm(best_params['pop_size'], best_params['num_generations'], best_params['mutation_rate'])

    print("Best route:", best_route)
    print("Best distance:", shortest_distance)
    

In [83]:
sigopt_optimization()

Best hyperparameters found:
Assignments({
  "mutation_rate": 0.2780458192949944,
  "num_generations": 235,
  "pop_size": 941
})
Best route: ['1', '5', '6', '9', '11']
Best distance: 23


### `ACO`

### ⊗ **Import libraries**: 

In [16]:
import numpy as np

### ⊗ **Define the connections from match**: 

In [17]:
# Origin Destiny Distance
# (1,      3,       4)
connections = {
    (1, 3): 4, (1, 5): 6, (1, 4): 4, (1, 7): 20,
    (3, 2): 4,
    (4, 2): 6, (4, 5): 4,
    (2, 5): 5,
    (5, 7): 8, (5, 6): 4,
    (6, 9): 5,
    (7, 8): 6, (7, 11): 15, (7, 9): 7,
    (8, 10): 7, (8, 9): 15,
    (9, 10): 10, (9, 11): 8,
    (10, 11): 6
}

unique_players = set([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

### ⊗ **Calculate the distance**: 

In [18]:
def distance(player1, player2):
    if (player1, player2) in connections:
        return connections[(player1, player2)]
    elif (player2, player1) in connections:
        return connections[(player2, player1)]
    elif player1 == player2:
        return 0
    else:
        return float('inf')  # Return a large value for non-connected connections
    
def total_distance(route):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance(route[i], route[i + 1])
    return total_distance

In [19]:
class Ant:
    def __init__(self, players, alpha, beta):

        self.players = players
        self.alpha = alpha
        self.beta = beta
        self.path = []  # Stores the path taken by the ant (list of players)
        self.visited = set()  # Set to keep track of visited players
        self.current_player = None  # The current player the ant is in
        self.unique_players = unique_players  # Set of unique player names

    def select_next_player(self, pheromone_matrix):
        unvisited_players = [player for player in self.unique_players if player not in self.visited and player != self.current_player]

        probabilities = []
        total = 0

        for player in unvisited_players:
            distance_to_player = distance(self.current_player, player)

            if(distance_to_player > 0):  # Two different players             
                pheromone = pheromone_matrix[self.current_player][player]
                probabilities.append((pheromone ** self.alpha) * (1.0 / distance_to_player) ** self.beta)
            else:
                probabilities.append(0)

            total += probabilities[-1]

        if(total > 0):
            probabilities = [p / total for p in probabilities]
        else:
            probabilities = [0 for p in probabilities]
        
        if(sum(probabilities) > 0.9):
            selected_player = np.random.choice(unvisited_players, size=1, p=probabilities)[0]
            self.path.append(selected_player)
            
            self.visited.add(selected_player)
            self.current_player = selected_player


In [20]:
def ant_colony_optimization(connections, num_ants, num_iterations, alpha=1.0, beta=5.0, evaporation=0.5):
    num_players = len(unique_players)
    pheromone_matrix = {player1: {player2: 1.0 for player2 in unique_players if player1 != player2} for player1 in unique_players}

    best_route = None
    best_distance = float('inf')  # Set an initial value for best_distance

    for _ in range(num_iterations):
        ants = [Ant(unique_players, alpha, beta) for _ in range(num_ants)]

        # Initialize each ant's current player 1
        for i, ant in enumerate(ants):
            ant.current_player = 1
            ant.path.append(1)
            ant.visited.add(ant.current_player)

        # Construct the tour for each ant
        for _ in range(num_players):
            for ant in ants:
                if ant.current_player != 11:
                    ant.select_next_player(pheromone_matrix)

        # Calculate the total distance of each ant's tour
        for ant in ants:
            ant_total_distance = total_distance(ant.path)
            if ant_total_distance < best_distance:  
                if ant.path[-1] == 11:        
                    best_route = ant.path.copy()
                    best_distance = ant_total_distance
                
        # Update pheromone levels
        for player1 in unique_players:
            for player2 in unique_players:
                if player1 != player2:
                    pheromone_matrix[player1][player2] *= (1 - evaporation)

        for ant in ants:
            if len(ant.path) >= 2:
                if(best_distance > 0):
                    pheromone_delta = 1.0 / best_distance
                    for i in range(len(ant.path) - 1):
                        player1, player2 = ant.path[i], ant.path[i + 1]                                     
                        pheromone_matrix[player1][player2] += pheromone_delta

    return best_route, best_distance

In [21]:
# Example usage:
num_ants = 50
num_iterations = 10

best_route, best_distance = ant_colony_optimization(connections, num_ants, num_iterations)

print("Best route:", best_route)
print("Best distance:", best_distance)

Best route: [1, 5, 6, 9, 11]
Best distance: 23


### ⊗ **Optimizing ACO algorithm using Intel® SigOpt**: 

In [33]:
import numpy as np
import sigopt

In [34]:
# Origin Destiny Distance
# (1,      3,       4)
connections = {
    (1, 3): 4, (1, 5): 6, (1, 4): 4, (1, 7): 20,
    (3, 2): 4,
    (4, 2): 6, (4, 5): 4,
    (2, 5): 5,
    (5, 7): 8, (5, 6): 4,
    (6, 9): 5,
    (7, 8): 6, (7, 11): 15, (7, 9): 7,
    (8, 10): 7, (8, 9): 15,
    (9, 10): 10, (9, 11): 8,
    (10, 11): 6
}


unique_players = set([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

In [35]:
def distance(player1, player2):
    if (player1, player2) in connections:
        return connections[(player1, player2)]
    elif (player2, player1) in connections:
        return connections[(player2, player1)]
    elif player1 == player2:
        return 0
    else:
        return float('inf')  # Return a large value for non-connected connections
    
def total_distance(route):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance(route[i], route[i + 1])
    return total_distance

In [36]:
class Ant:
    def __init__(self, players, alpha, beta):

        self.players = players
        self.alpha = alpha
        self.beta = beta
        self.path = []  # Stores the path taken by the ant (list of players)
        self.visited = set()  # Set to keep track of visited players
        self.current_player = None  # The current player the ant is in
        self.unique_players = unique_players  # Set of unique player names

    def select_next_player(self, pheromone_matrix):
        unvisited_players = [player for player in self.unique_players if player not in self.visited and player != self.current_player]

        probabilities = []
        total = 0

        for player in unvisited_players:
            distance_to_player = distance(self.current_player, player)

            if(distance_to_player > 0):  # Two different players             
                pheromone = pheromone_matrix[self.current_player][player]
                probabilities.append((pheromone ** self.alpha) * (1.0 / distance_to_player) ** self.beta)
            else:
                probabilities.append(0)

            total += probabilities[-1]

        if(total > 0):
            probabilities = [p / total for p in probabilities]
        else:
            probabilities = [0 for p in probabilities]
        
        if(sum(probabilities) > 0.9):
            selected_player = np.random.choice(unvisited_players, size=1, p=probabilities)[0]
            self.path.append(selected_player)
            
            self.visited.add(selected_player)
            self.current_player = selected_player


In [37]:
def ant_colony_optimization(connections, num_ants, num_iterations, alpha, beta, evaporation):
    num_players = len(unique_players)
    pheromone_matrix = {player1: {player2: 1.0 for player2 in unique_players if player1 != player2} for player1 in unique_players}

    best_route = None
    best_distance = float('inf')  # Set an initial value for best_distance

    for _ in range(num_iterations):
        ants = [Ant(unique_players, alpha, beta) for _ in range(num_ants)]

        # Initialize each ant's current player 1
        for i, ant in enumerate(ants):
            ant.current_player = 1
            ant.path.append(1)
            ant.visited.add(ant.current_player)

        # Construct the tour for each ant
        for _ in range(num_players):
            for ant in ants:
                if ant.current_player != 11:
                    ant.select_next_player(pheromone_matrix)

        # Calculate the total distance of each ant's tour
        for ant in ants:
            ant_total_distance = total_distance(ant.path)
            if ant_total_distance < best_distance:  
                if ant.path[-1] == 11:        
                    best_route = ant.path.copy()
                    best_distance = ant_total_distance
                
        # Update pheromone levels
        for player1 in unique_players:
            for player2 in unique_players:
                if player1 != player2:
                    pheromone_matrix[player1][player2] *= (1 - evaporation)

        for ant in ants:
            if len(ant.path) >= 2:
                if(best_distance > 0):
                    pheromone_delta = 1.0 / best_distance
                    for i in range(len(ant.path) - 1):
                        player1, player2 = ant.path[i], ant.path[i + 1]                                     
                        pheromone_matrix[player1][player2] += pheromone_delta

    return best_route, best_distance

In [38]:
def evaluate_ant_colony(params):
    num_ants = params['num_ants']
    num_iterations = params['num_iterations']
    alpha = params['alpha']
    beta = params['beta']
    evaporation = params['evaporation']
    
    # Call the ant_colony_optimization function and return the result you want to optimize
    best_route, best_distance = ant_colony_optimization(connections, num_ants, num_iterations, alpha, beta, evaporation)
    
    return best_distance

def sigopt_optimization():
    # Set your SigOpt API TOKEN or use generic API TOKEN
    SIGOPT_API_TOKEN = 'MJZRDBRNTMOPVYPCTTMRAMDIQDUEDTDJGOUMPFSIQTRKLYJJ'

    # Configure the SigOpt client
    conn = sigopt.Connection(client_token=SIGOPT_API_TOKEN)

    # Define the variables to be optimized and their ranges
    param_space = [
        {'name': 'num_ants', 'type': 'int', 'bounds': {'min': 5, 'max': 50}},
        {'name': 'num_iterations', 'type': 'int', 'bounds': {'min': 10, 'max': 500}},
        {'name': 'alpha', 'type': 'double', 'bounds': {'min': 0.1, 'max': 2.0}},
        {'name': 'beta', 'type': 'double', 'bounds': {'min': 0.1, 'max': 2.0}},
        {'name': 'evaporation', 'type': 'double', 'bounds': {'min': 0.1, 'max': 0.9}},
    ]

    # Create an experiment on SigOpt
    experiment = conn.experiments().create(
        name='Robots ACO Optimization',
        parameters=param_space,
        metrics=[{'name': 'best_distance', 'objective': 'minimize'}]
    )

    # Run the optimization for a number of iterations
    for _ in range(30):
        suggestion = conn.experiments(experiment.id).suggestions().create()
        suggestion_id = suggestion.id
        
        # Get the suggested hyperparameters
        suggested_params = suggestion.assignments
        
        # Evaluate the performance with the suggested hyperparameters
        result = evaluate_ant_colony(suggested_params)
        
        # Report the results to SigOpt
        conn.experiments(experiment.id).observations().create(
            suggestion=suggestion_id,
            value=result,
        )

    # Get the best optimization results
    best_assignments = conn.experiments(experiment.id).best_assignments().fetch()

    # Print the best-found hyperparameters
    print("Best hyperparameters found:")
    print(best_assignments.data[0].assignments)

    # Run the ACO with the best-found hyperparameters
    best_params = best_assignments.data[0].assignments
    best_route, best_distance = ant_colony_optimization(connections, best_params['num_ants'], best_params['num_iterations'], best_params['alpha'], best_params['beta'], best_params['evaporation'])

    # Print the best route and distance found
    print("Best route:", best_route)
    print("Best distance:", best_distance)

In [39]:
sigopt_optimization()

Best hyperparameters found:
Assignments({
  "alpha": 1.209943535489275,
  "beta": 0.19761079490970704,
  "evaporation": 0.1,
  "num_ants": 5,
  "num_iterations": 352
})
Best route: [1, 5, 6, 9, 11]
Best distance: 23


### `PSO`

In [1]:
import numpy as np
import random
import copy
import sigopt

# Define as conexões dos jogadores
player_connections = {
    1: {3: 4, 5: 6, 4: 4, 7: 20},
    2: {5: 5},
    3: {2: 4},
    4: {2: 6, 5: 4},
    5: {6: 4, 7: 8},
    6: {9: 5},
    7: {8: 6, 11: 15, 9: 7},
    8: {10: 7, 9: 15},
    9: {10: 10, 11: 8},
    10: {11: 6}
}

In [2]:
class Particle:
    def __init__(self, player_connections):
        # Generate random route, starting with 1 and finishing with 11.
        players = list(player_connections.keys())
        players.remove(1)
        players = players + [11]
        random.shuffle(players)
        self.position = [1] + players
        self.best_position = self.position
        self.best_fitness = fitness(self.position, player_connections)

In [3]:
def fitness(route, connections):
    total_distance = 0
    for i in range(len(route)):
        # If route[i] is 11, the route has already finished
        if route[i] == 11:
            return total_distance
        if route[i+1] in connections[route[i]]:
            total_distance += connections[route[i]][route[i+1]]
        else:
            # If the connection does not exist, we can set it to infinite.
            total_distance += float('inf')
            
def distance(city1, city2, connections):
    if city1 == 11:
      return 1
    if city2 in connections[city1]:
        return connections[city1][city2]
    else:
        # If the connection does not exist, we can set it to infinite.
        return float('inf')

def heuristic_crossover(x1, x2, player_connections):
    n = len(x1) - 1

    # Select players randomly who can receive the ball from the starting player.
    v = random.choice(list(player_connections.get(1, {}).keys()))

    # Insert first player with the randomly choosen.
    x = [1] + [v]

    x1.remove(v)
    x1.insert(1, v)
    x2.remove(v)
    x2.insert(1, v)
    i = 2
    j = 2

    while i <= n and j <= n:

        if x1[i] in x and x2[j] in x:
            i += 1
            j += 1
        elif x1[i] in x:
            x.append(x2[j])
            j += 1
        elif x2[j] in x:
            x.append(x1[i])
            i += 1
        else:
            u = x[-1]
            dist_1 = distance(u, x1[i], player_connections)
            dist_2 = distance(u, x2[j], player_connections)
            if dist_1 < dist_2:
                x.append(x1[i])
                i += 1
            else:
                x.append(x2[j])
                j += 1
    return x

In [4]:
def PSO(connections, n_particles, n_iterations):
    # Creating particles
    particles = [Particle(connections) for _ in range(n_particles)]

    # Initializing global_best_position and global_best_fitness
    global_best_position = particles[0].position
    global_best_fitness = particles[0].best_fitness
    aha = global_best_position
    for _ in range(n_iterations):
        # Checking each route
        for particle in particles:
            current_fitness = fitness(particle.position, connections)

            # Update pbest
            if current_fitness < particle.best_fitness:
                particle.best_fitness = current_fitness
                particle.best_position = particle.position
            # Update gbest
            if current_fitness < global_best_fitness:
                global_best_fitness = current_fitness
                global_best_position = particle.position
                tome = global_best_position

        # Update each route, using heuristic crossover (HC)
        for particle in particles:
            # Calculate new route using heuristic crossover with pbest and gbest
            particle.position = heuristic_crossover(copy.deepcopy(particle.best_position), copy.deepcopy(global_best_position), connections)
    return global_best_position, global_best_fitness


In [5]:
n_particles = 200
n_iterations = 1000
best_route, best_fitness = PSO(player_connections, n_particles, n_iterations)

print("Best route:", end=' ')
for i in best_route:
  print(i, end=' ')
  if i == 11:
    break
print("\nBest distance:", best_fitness)

Best route: 1 5 7 9 11 
Best distance: 29


### ⊗ **Optimizing PSO algorithm using Intel® SigOpt**: 

In [6]:
import numpy as np
import random
import copy
import sigopt

# Define connections
player_connections = {
    1: {3: 4, 5: 6, 4: 4, 7: 20},
    2: {5: 5},
    3: {2: 4},
    4: {2: 6, 5: 4},
    5: {6: 4, 7: 8},
    6: {9: 5},
    7: {8: 6, 11: 15, 9: 7},
    8: {10: 7, 9: 15},
    9: {10: 10, 11: 8},
    10: {11: 6}
}

In [7]:
class Particle:
    def __init__(self, player_connections):
        # Generate random route, starting with 1 and finishing with 11.
        players = list(player_connections.keys())
        players.remove(1)
        players = players + [11]
        random.shuffle(players)
        self.position = [1] + players
        self.best_position = self.position
        self.best_fitness = fitness(self.position, player_connections)

def fitness(route, connections):
    total_distance = 0
    for i in range(len(route)):
        # If route[i] is 11, the route has already finished
        if route[i] == 11:
            return total_distance
        if route[i+1] in connections[route[i]]:
            total_distance += connections[route[i]][route[i+1]]
        else:
            # If the connection does not exist, we can set it to infinite.
            total_distance += float('inf')

def distance(city1, city2, connections):
    if city1 == 11:
        return 1
    if city2 in connections[city1]:
        return connections[city1][city2]
    else:
        # If the connection does not exist, we can set it to infinite.
        return float('inf')

def heuristic_crossover(x1, x2, player_connections):
    n = len(x1) - 1

    # Select players randomly who can receive the ball from the starting player.
    v = random.choice(list(player_connections.get(1, {}).keys()))

    # Insert first player with the randomly chosen.
    x = [1] + [v]

    x1.remove(v)
    x1.insert(1, v)
    x2.remove(v)
    x2.insert(1, v)
    i = 2
    j = 2

    while i <= n and j <= n:

        if x1[i] in x and x2[j] in x:
            i += 1
            j += 1
        elif x1[i] in x:
            x.append(x2[j])
            j += 1
        elif x2[j] in x:
            x.append(x1[i])
            i += 1
        else:
            u = x[-1]
            dist_1 = distance(u, x1[i], player_connections)
            dist_2 = distance(u, x2[j], player_connections)
            if dist_1 < dist_2:
                x.append(x1[i])
                i += 1
            else:
                x.append(x2[j])
                j += 1
    return x

In [8]:
def PSO(connections, n_particles, n_iterations):
    # Creating particles
    particles = [Particle(connections) for _ in range(n_particles)]

    # Initializing global_best_position and global_best_fitness
    global_best_position = particles[0].position
    global_best_fitness = particles[0].best_fitness
    aha = global_best_position
    for _ in range(n_iterations):
        # Checking each route
        for particle in particles:
            current_fitness = fitness(particle.position, connections)

            # Update pbest
            if current_fitness < particle.best_fitness:
                particle.best_fitness = current_fitness
                particle.best_position = particle.position
            # Update gbest
            if current_fitness < global_best_fitness:
                global_best_fitness = current_fitness
                global_best_position = particle.position
                tome = global_best_position

        # Update each route, using heuristic crossover (HC)
        for particle in particles:
            # Calculate new route using heuristic crossover with pbest and gbest
            particle.position = heuristic_crossover(copy.deepcopy(particle.best_position), copy.deepcopy(global_best_position), connections)
    
    # Extract the best route
    best_route = global_best_position
    return best_route

def evaluate_route(params):
    n_particles = params['n_particles']
    n_iterations = params['n_iterations']
    best_route = PSO(player_connections, n_particles, n_iterations)
    return fitness(best_route, player_connections)


In [ ]:
def sigopt_optimization():
    # Set your SigOpt API TOKEN here
    SIGOPT_API_TOKEN = 'YOUR_SIGOPT_API_TOKEN'

    # Configure the SigOpt client
    conn = sigopt.Connection(client_token="MJZRDBRNTMOPVYPCTTMRAMDIQDUEDTDJGOUMPFSIQTRKLYJJ")

    # Define the optimization parameters and their ranges
    param_space = [
        {'name': 'n_particles', 'type': 'int', 'bounds': {'min': 100, 'max': 300}},
        {'name': 'n_iterations', 'type': 'int', 'bounds': {'min': 500, 'max': 1500}}
    ]

    # Create an experiment on SigOpt
    experiment = conn.experiments().create(
        name='PSO Optimization',
        parameters=param_space,
        metrics=[{'name': 'best_distance', 'objective': 'minimize'}]
    )

    # Run the optimization for a number of iterations
    for _ in range(30):
        suggestion = conn.experiments(experiment.id).suggestions().create()
        suggestion_id = suggestion.id

        # Get the suggested hyperparameters
        suggested_params = suggestion.assignments

        # Evaluate the performance with the suggested hyperparameters
        result = evaluate_route(suggested_params)

        # Report the results to SigOpt
        conn.experiments(experiment.id).observations().create(
            suggestion=suggestion_id,
            value=result
        )

    # Get the best optimization results
    best_assignments = conn.experiments(experiment.id).best_assignments().fetch()

    # Print the best-found hyperparameters
    print("Best hyperparameters found:")
    print(best_assignments.data[0].assignments)

    # Run the PSO with the best-found hyperparameters
    best_params = best_assignments.data[0].assignments
    best_route = PSO(player_connections, best_params['n_particles'], best_params['n_iterations'])

    # Print the best route and distance found
    print("Best route:", end=' ')
    for i in best_route:
        print(i, end=' ')
        if i == 11:
            break
    print("\nBest distance:", fitness(best_route, player_connections))

# Run the SigOpt optimization
sigopt_optimization()